# Baseline model

In [1]:
import numpy as np
import pandas as pd

### Opening data sets:

In [2]:
recipes = pd.read_csv("../raw_data/recipes.csv")

In [3]:
recipes.head()

,id,name,ingredients,ingredient_ids
0,137739,arriba baked winter squash mexican style,"['winter squash', 'mexican seasoning', 'mixed ...","[7933, 4694, 4795, 3723, 840, 5006, 6270]"
1,31490,a bit different breakfast pizza,"['prepared pizza crust', 'sausage patty', 'egg...","[5481, 6324, 2499, 4717, 6276, 1170]"
2,59389,alouette potatoes,"['spreadable cheese with garlic and herbs', 'n...","[1170, 4918, 6426, 5185, 7099, 5006, 6009, 627..."
3,5289,apple a day milk shake,"['milk', 'vanilla ice cream', 'frozen apple ju...","[4717, 7474, 2946, 150]"
4,25274,aww marinated olives,"['fennel seeds', 'green olives', 'ripe olives'...","[2587, 3437, 5002, 3184, 5324, 5068, 5058, 131..."


In [4]:
# has all ingredients, incluing the repeated ones
ingredients = pd.read_csv("../raw_data/ingredient_keys.csv")[['id','replaced','count']].drop_duplicates()

In [5]:
ingredients.head()

,id,replaced,count
0,4308,lettuce,4507
42,2744,french vanilla pudding and pie filling mix,5
43,6843,stove top stuffing mix,114
45,1910,cream cheese,9915
86,1168,cheddar,13304


In [6]:
ordered_keys = ingredients.sort_values(by='id').reset_index(drop=True)

In [7]:
### descomentar se quiser criar o csv ordered_keys
# ordered_keys.to_csv('../raw_data/ordered_keys.csv', index=False)

In [8]:
ordered_keys

,id,replaced,count
0,0,'s baking chocolate,2
1,1,'s chocolate chip,13
2,2,'s hugs chocolate,7
3,3,'s sauce,6
4,4,'s sour cream,2
...,...,...,...
8018,8018,ziploc bag,32
8019,8019,ziti pastum,200
8020,8020,ziti rigatus,2
8021,8021,zucchini,3948


# Buildind the base df "ingredient_pairs"

In [9]:
# ingredient_pairs = pd.DataFrame(columns = list(ingredients['id'].sort_values()),index=list(ingredients['id'].sort_values()), data=0)

In [10]:
# ingredient_pairs

# Creating loop to fill df with pair combinations

In [11]:
### descomentar se quiser rodar a célula de baixo
# recipes_list = recipes['ingredient_ids'].to_numpy()
# recipes_list = np.array([ast.literal_eval(item) for item in recipes_list])

In [12]:
#### descomentar a de cima e essa se quiser executar de novo a criação da combinação dos ingredientes
# import ast
# import time
# %%time
# for ingr_id in range(8023):
#     print(f'looking for recipes containing ingredient {ingr_id}', end='\r')
#     for idx, recipe_ingredients in enumerate(recipes_list):
#         if ingr_id in recipe_ingredients:
#             ingredient_pairs.loc[ingr_id,recipe_ingredients] += 1
#             recipe_ingredients.remove(ingr_id)
#             if not recipe_ingredients:
#                 np.delete(recipes_list, idx, 0)

In [13]:
#### descomentar se quiser criar o csv
# ingredient_pairs.to_csv('../raw_data/ingredient_pairs.csv', index=False)

In [14]:
ingredient_pairs = pd.read_csv('../raw_data/ingredient_pairs.csv')
ingredient_pairs

,0,1,2,3,4,5,6,7,8,9,...,8013,8014,8015,8016,8017,8018,8019,8020,8021,8022
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,13,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,29,0,0,1,0
8019,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,187,0,8,0
8020,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8021,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3661,0


# Mirror the matrix

In [15]:
# mirrorring loop: gets the vertical component of the item, horizontal component, 
# and attibutes it to the rows in the ingredient_pairs df
for idx in range(8023):
    vertical = ingredient_pairs.iloc[0:int(idx+1),int(idx):int(idx+1)].T.values.tolist()[0]
    horizontal = ingredient_pairs.iloc[idx:idx+1,idx+1:].values.tolist()[0]
    ingredient_pairs.iloc[idx:(idx+1),:] = pd.DataFrame(vertical + horizontal).T

In [16]:
ingredient_pairs

,0,1,2,3,4,5,6,7,8,9,...,8013,8014,8015,8016,8017,8018,8019,8020,8021,8022
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,13,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,29,0,0,1,0
8019,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,187,0,8,0
8020,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8021,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,1,8,0,3661,0


# Dividing the pairs array by the single ingredient vector

In [17]:
# vector with number of recipes for each ingredient
vector = np.diagonal(ingredient_pairs)

In [18]:
# ingredients with no recipes
np.where(vector == 0)[0]

array([  24,   67,  404,  809, 1071, 1092, 1271, 2691, 3017, 3751, 3847,
       3995, 4036, 4450, 4542, 4629, 4802, 5135, 5310, 5385, 5396, 5572,
       5606, 6347, 6396, 6482, 6656, 7403, 7664, 7842])

In [19]:
ingredient_pairs_np = np.array(ingredient_pairs)

In [20]:
combination_percentage = ingredient_pairs_np / vector[:,None]

<ipython-input-20-0735a442b740>:1: RuntimeWarning: invalid value encountered in true_divide
  combination_percentage = ingredient_pairs_np / vector[:,None]


In [21]:
combination_percentage = pd.DataFrame(combination_percentage)

In [22]:
combination_percentage.isnull().sum().sum()

240690

In [23]:
# #### descomentar se quiser criar o csv
# combination_percentage.to_csv('../raw_data/combination_percentage_01.csv', index=False)

# Code to send to front-end

In [24]:
# pegar do csv
combination_percentage

,0,1,2,3,4,5,6,7,8,9,...,8013,8014,8015,8016,8017,8018,8019,8020,8021,8022
0,1.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
1,0.0,1.000000,0.076923,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
2,0.0,0.166667,1.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
3,0.0,0.000000,0.000000,1.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
4,0.0,0.000000,0.000000,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8018,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,1.000000,0.000000,0.0,0.034483,0.0
8019,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,1.000000,0.0,0.042781,0.0
8020,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,1.0,0.000000,0.0
8021,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000273,0.0,0.0,...,0.0,0.0,0.000273,0.0,0.0,0.000273,0.002185,0.0,1.000000,0.0


In [25]:
# precisa também do csv da ordered keys

In [26]:
# def get_combinations(index):
    
#     ingredient_row = combination_percentage.iloc[index:index+1,:].T.sort_values(by=index, ascending=False)
#     top_results = []
#     for index in range(1,20):
#         ingr_id = ingredient_row.T.columns[index]
#         name = ordered_keys['replaced'].iloc[ingr_id]
#         top_results.append(name)

#     return top_results

# trying to input written word on this function (search_word)
def get_combinations(search_word):
    word_index = ordered_keys[ordered_keys['replaced'] == search_word].index[0]
    ingredient_row = combination_percentage.iloc[word_index:word_index+1,:].T.sort_values(by=word_index, ascending=False)
    size_results = ingredient_row[ingredient_row[word_index] != 0].shape[0]
    top_results = []
    for index in range(1,20):
        ingr_id = ingredient_row.T.columns[index]
        name = ordered_keys['replaced'].iloc[ingr_id]
        top_results.append(name)

    return top_results

In [27]:
get_combinations('shrimp')

['salt',
 'olive oil',
 'garlic clove',
 'butter',
 'onion',
 'garlic',
 'scallion',
 'salt and pepper',
 'lemon juice',
 'water',
 'parmesan cheese',
 'tomato',
 'soy sauce',
 'pepper',
 'celery',
 'parsley',
 'cayenne pepper',
 'fresh parsley',
 'mayonnaise']

In [28]:
def surprise(search_word):
    word_index = ordered_keys[ordered_keys['replaced'] == search_word].index[0]
    ingredient_row = combination_percentage.iloc[word_index:word_index+1,:].T.sort_values(by=word_index, ascending=False)
    size_results = ingredient_row[ingredient_row[word_index] != 0].shape[0]
    top_results = []
    for index in np.random.randint(0,int(size_results/3),20):
        ingr_id = ingredient_row.T.columns[index]
        name = ordered_keys['replaced'].iloc[ingr_id]
        top_results.append(name)

    return top_results

In [29]:
surprise('shrimp')

['whipping cream',
 'white wine',
 'spanish onion',
 'gingerroot',
 'mayonnaise',
 'roasted peanut',
 'sesame oil',
 'lime zest',
 'garlic bread',
 'cold water',
 'orange zest',
 'white sugar',
 'snow pea',
 'heavy cream',
 'lemon peel',
 'asparagus spear',
 'yellow onion',
 'flat leaf parsley',
 'whipping cream',
 'cajun seasoning']

# Matriz para algoritmo de recomendações (Co-occurrence)

In [30]:
co_occurrence = ingredient_pairs_np

In [31]:
# para seguir o passo a passo do texto da songxia-sophia
np.fill_diagonal(co_occurrence, 0)

In [32]:
co_occurrence

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [33]:
co_occurrence = pd.DataFrame(co_occurrence)
co_occurrence

,0,1,2,3,4,5,6,7,8,9,...,8013,8014,8015,8016,8017,8018,8019,8020,8021,8022
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
8019,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,8,0
8020,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8021,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,1,8,0,0,0


In [34]:
# ### descomentar quando quiser criar novo csv
# co_occurrence.to_csv('../raw_data/co_occurrence.csv', index=False)

In [35]:
from scipy.sparse import csr_matrix

co_occurrence_sparse = csr_matrix(co_occurrence)

In [36]:
# from scipy.sparse import save_npz

# save_npz('../raw_data/co_occurrence_sparse.npz', co_occurrence_sparse)

# Creating info for the presentation

In [37]:
vector

array([   1,   13,    6, ...,    1, 3661,    2])

In [39]:
# Sort in ascending order
sorted_array = np.sort(vector)

In [40]:
# Reverse the sorted array
reverse_array = sorted_array[::-1]

In [41]:
reverse_array

array([66482, 43952, 39039, ...,     0,     0,     0])

In [42]:
df_vector = pd.DataFrame(vector)

In [51]:
df_top_ingredients = df_vector.nlargest(30, 0)

In [52]:
df_top_ingredients

,0
6270,66482
840,43952
5010,39039
2499,38703
6906,33884
5006,31607
3203,27218
7655,25953
4717,22053
2683,20770


In [56]:
top_ids_list = df_top_ingredients.index.values.tolist()

In [57]:
top_ids_list

[6270,
 840,
 5010,
 2499,
 6906,
 5006,
 3203,
 7655,
 4717,
 2683,
 5319,
 800,
 3184,
 332,
 6335,
 6276,
 63,
 1168,
 5180,
 335,
 4253,
 7449,
 7557,
 1093,
 6654,
 590,
 1511,
 7213,
 6696,
 1910]

In [59]:
# def get_combinations(search_word):
#     word_index = ordered_keys[ordered_keys['replaced'] == search_word].index[0]
#     ingredient_row = combination_percentage.iloc[word_index:word_index+1,:].T.sort_values(by=word_index, ascending=False)
#     size_results = ingredient_row[ingredient_row[word_index] != 0].shape[0]
#     top_results = []
#     for index in range(1,20):
#         ingr_id = ingredient_row.T.columns[index]
#         name = ordered_keys['replaced'].iloc[ingr_id]
#         top_results.append(name)

#     return top_results
top_ingredients_list = []
for item in top_ids_list:
    itemname = ordered_keys['replaced'].iloc[item]
    top_ingredients_list.append(itemname)
top_ingredients_list

['salt',
 'butter',
 'onion',
 'egg',
 'sugar',
 'olive oil',
 'garlic clove',
 'water',
 'milk',
 'flmy',
 'pepper',
 'brown sugar',
 'garlic',
 'baking powder',
 'scallion',
 'salt and pepper',
 'all-purpose flmy',
 'cheddar',
 'parmesan cheese',
 'baking soda',
 'lemon juice',
 'vanilla',
 'vegetable oil',
 'carrot',
 'sour cream',
 'black pepper',
 'cinnamon',
 'tomato',
 'soy sauce',
 'cream cheese']

In [60]:
df_top_ingredients['Names'] = top_ingredients_list

In [61]:
df_top_ingredients

,0,Names
6270,66482,salt
840,43952,butter
5010,39039,onion
2499,38703,egg
6906,33884,sugar
5006,31607,olive oil
3203,27218,garlic clove
7655,25953,water
4717,22053,milk
2683,20770,flmy


In [38]:
co_occurrence

,0,1,2,3,4,5,6,7,8,9,...,8013,8014,8015,8016,8017,8018,8019,8020,8021,8022
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
8019,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,8,0
8020,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8021,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,1,8,0,0,0
